In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [6]:
df.dropna(inplace=True)

In [7]:
import re

def clean_up_tweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove retweets:
    txt = re.sub(r'RT : ', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt).strip()
    return txt

In [8]:
df["text"] = df["text"].apply(clean_up_tweet)

In [ ]:
df

In [ ]:
df.drop(["textID", "selected_text"], inplace=True, axis=1)

In [29]:
TARGET = "sentiment"

In [12]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
def lemmatize_insults(tweet):
    return re.sub(r"(\*){3,}\*", "-INSULT-", tweet).strip()

df["text"] = df["text"].progress_apply(lemmatize_insults)
df["text"]

In [ ]:
def expand_contractions(phrase):
    phrase = re.sub(r"\`", "\'", phrase)
    
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase.strip()

df["text"] = df["text"].progress_apply(expand_contractions)
df["text"]

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text).strip()

df["text"] = df["text"].progress_apply(remove_punctuation)
df["text"]

In [ ]:
def remove_numbers(tweet):
    return re.sub(r'\w*\d\w*', "", tweet).strip()

df["text"] = df["text"].progress_apply(remove_numbers)
df["text"]

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
def lemmatize_tweet(tweet):
    lemmatized_text = nlp(tweet)
    lemmatized_text_lst = [word.lemma_ for word in lemmatized_text]
    return " ".join(lemmatized_text_lst)

In [ ]:
df["text"] = df["text"].progress_apply(lemmatize_tweet)

In [ ]:
df["text"]

In [16]:
# df.to_csv("lemmatized_df.csv")
df = pd.read_csv('lemmatized_df.csv', index_col=0).dropna()
df

,text,sentiment
0,-PRON- would have respond if -PRON- be go,neutral
1,Sooo SAD -PRON- will miss -PRON- here in San D...,negative
2,-PRON- boss be bully -PRON-,negative
3,what interview leave -PRON- alone,negative
4,son of INSULT why could not -PRON- put -PRON- ...,negative
...,...,...
27476,wish -PRON- could come see u on Denver husba...,negative
27477,-PRON- have wonder about rake to the client ...,negative
27478,Yay good for both of -PRON- enjoy the break ...,positive
27479,but -PRON- be worth -PRON- INSULT,positive


In [17]:
from string import punctuation

def remove_punctuation(tweet):
    tweet = re.sub(f"[{punctuation}]", "", tweet).strip()
    return tweet

df["text"] = df["text"].progress_apply(remove_punctuation)

100%|██████████| 27476/27476 [00:00<00:00, 294572.61it/s]


In [20]:
df["text"]

0                    PRON would have respond if PRON be go
1           Sooo SAD PRON will miss PRON here in San Diego
2                                  PRON boss be bully PRON
3                          what interview leave PRON alone
4        son of INSULT why could not PRON put PRON on t...
                               ...                        
27476    wish PRON could come see u on Denver   husband...
27477    PRON have wonder about rake to   the client ha...
27478    Yay good for both of PRON enjoy the break   PR...
27479                      but PRON be worth PRON   INSULT
27480    all this flirting go on   the ATG smile Yay   hug
Name: text, Length: 27476, dtype: object

In [21]:
np.unique(df.loc[20, "text"].split(), return_counts=True)

(array(['PRON', 'be', 'find', 'hope', 'marly', 'oh', 'so', 'soon', 'sorry'],
       dtype='<U5'),
 array([4, 1, 1, 1, 1, 1, 1, 1, 1]))

In [22]:
import nltk

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielsiles/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), ngram_range=(1,3), min_df=25, max_df=0.5)

In [25]:
def sentiment_encoder(sentiment):
    if sentiment == 'negative':
        return 0
    elif sentiment == 'neutral':
        return 1
    else:
        return 2

df[TARGET] = df[TARGET].apply(sentiment_encoder)

NameError: name 'TARGET' is not defined

In [ ]:
!pip install TextBlob

In [ ]:
from textblob import TextBlob
# from wordcloud import WordCloud

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
    return  TextBlob(text).sentiment.polarity


# Create two new columns 'Subjectivity' & 'Polarity'
df['Subjectivity'] = df['text'].apply(getSubjectivity)
df['Polarity'] = df['text'].apply(getPolarity)

In [ ]:
df

In [30]:
X = df.drop(TARGET, axis=1)
y = df[TARGET]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (24728, 1)
X_test shape: (2748, 1)


In [32]:
X_train_vectorized = vectorizer.fit_transform(X_train["text"])
X_test_vectorized = vectorizer.transform(X_test["text"])

In [33]:
print("X_train_vectorized shape:", X_train_vectorized.shape)
print("X_test_vectorized shape:", X_test_vectorized.shape)

X_train_vectorized shape: (24728, 1419)
X_test_vectorized shape: (2748, 1419)


In [34]:
X_train_t = pd.concat([pd.DataFrame(X_train_vectorized.toarray(), index=X_train.index), X_train.drop("text",axis=1)], axis=1)
X_test_t = pd.concat([pd.DataFrame(X_test_vectorized.toarray(), index=X_test.index), X_test.drop("text",axis=1)], axis=1)

In [35]:
print("X_train shape:", X_train_t.shape)
print("X_test shape:", X_test_t.shape)

X_train shape: (24728, 1419)
X_test shape: (2748, 1419)


In [37]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

mcc = OneVsRestClassifier(SVC(verbose=1))
mcc.fit(X_train_t, y_train)
y_pred = mcc.predict(X_test_t)

[LibSVM]

KeyboardInterrupt: 

In [ ]:
print("Score:", accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
mcc = OneVsRestClassifier(GaussianNB())
mcc.fit(X_train_t, y_train)
y_pred = mcc.predict(X_test_t)

In [ ]:
print("Score:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
np.unique(y)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC()
svc.fit(X_train_vectorized, y_train)

In [ ]:
y_pred = svc.predict(X_test_vectorized)

In [ ]:
from sklearn.metrics import accuracy_score

print("Score:", accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lin_reg = LogisticRegression(penalty="l1", solver="saga", C=10, max_iter=100000)
lin_reg.fit(X_train_vectorized, y_train)

In [ ]:
y_pred = lin_reg.predict(X_test_vectorized)

In [ ]:
from sklearn.metrics import accuracy_score

print("Score:", accuracy_score(y_test, y_pred))